# Etape 4 : Run Inference

In [11]:
import torch
print("Torch OK :", torch.__version__)
print("CUDA dispo ?", torch.cuda.is_available())

Torch OK : 2.9.1+cpu
CUDA dispo ? False


In [12]:
import os

# Se placer à la racine du repo (un niveau au-dessus de notebooks/)
os.chdir("..")
print("Nouveau CWD =", os.getcwd())

Nouveau CWD = c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01


In [4]:
# Chemins des fichiers
VIDEO_CSV = "data/splits/video_folds_5fold.csv"
FRAMES_ROOT = "data/frames/5_forstep1and2"
SUPERIMG_CSV = "data/splits/super_images_3x3_folds.csv"
SUPERIMG_ROOT = "data/super_images"

# Dossiers des modèles entraînés
FRAMEWISE_DIR = "exp/results/framewise_resnet18"
CONV_DIR = "exp/results/super_images_convnext"
TRES_DIR = "exp/results/super_images_tresnetxl"

print("📁 Chemins chargés :")
for name, val in {
    "VIDEO_CSV": VIDEO_CSV,
    "FRAMES_ROOT": FRAMES_ROOT,
    "SUPERIMG_CSV": SUPERIMG_CSV,
    "SUPERIMG_ROOT": SUPERIMG_ROOT,
    "FRAMEWISE_DIR": FRAMEWISE_DIR,
    "CONV_DIR": CONV_DIR,
    "TRES_DIR": TRES_DIR,
}.items():
    print(f"- {name}: {val}")


📁 Chemins chargés :
- VIDEO_CSV: data/splits/video_folds_5fold.csv
- FRAMES_ROOT: data/frames/5_forstep1and2
- SUPERIMG_CSV: data/splits/super_images_3x3_folds.csv
- SUPERIMG_ROOT: data/super_images
- FRAMEWISE_DIR: exp/results/framewise_resnet18
- CONV_DIR: exp/results/super_images_convnext
- TRES_DIR: exp/results/super_images_tresnetxl


In [12]:
import subprocess, sys

def run_python(script, args):
    cmd = [sys.executable, script] + args
    print("▶️ Commande :", " ".join(cmd))
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("❌ ERREUR (code", result.returncode, ")")
        print(result.stderr)
    print("\n--- FIN DU SCRIPT ---\n")
    return result

## Inference framewise (5 folds)

In [24]:
!python src/infer/framewise_infer.py \
  --video_csv data/splits/video_folds_5fold.csv \
  --frames_root data/frames/5_forstep1and2 \
  --ckpt exp/results/framewise_resnet18/resnet18_fold4/best_model.pth \
  --fold 4 \
  --out exp/results/framewise_resnet18/preds_resnet18_fold4.npz

Device: cpu
Fold 4 → 1291 vidéos
Loading checkpoint: exp/results/framewise_resnet18/resnet18_fold4/best_model.pth
Backbone: resnet18
[WARN] Aucune frame trouvée pour la vidéo '25232', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23916_23', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo '26985', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23921_9', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23917_12', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23906_10', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23921_11', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo '25232', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo 'd23916_23', utilisation de frames noires dummy.
[WARN] Aucune frame trouvée pour la vidéo '26985', utilisatio


Inference:   0%|          | 0/81 [00:00<?, ?it/s]c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)

Inference: 100%|██████████| 81/81 [03:02<00:00,  2.25s/it]


## Inference ConvNeXt (5 folds)

In [2]:
import subprocess
from pathlib import Path

# ⛔ Peu importe ton CWD actuel, on fixe la racine du projet ici :
PROJECT_ROOT = Path(r"c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier")

print("PROJECT_ROOT :", PROJECT_ROOT)
print("PROJECT_ROOT existe ?", PROJECT_ROOT.exists())

script_path = PROJECT_ROOT / "src" / "infer" / "superimages_infer.py"
splits_csv  = PROJECT_ROOT / "data" / "splits" / "super_images_3x3_folds.csv"
img_root    = PROJECT_ROOT / "data" / "super_images"
ckpt_root   = PROJECT_ROOT / "exp" / "results" / "super_images_convnext"

print("Script existe ?", script_path.exists())
print("CSV existe ?", splits_csv.exists())
print("Dossier images existe ?", img_root.exists())
print("Dossier ckpt existe ?", ckpt_root.exists())



PROJECT_ROOT : c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier
PROJECT_ROOT existe ? True
Script existe ? True
CSV existe ? True
Dossier images existe ? True
Dossier ckpt existe ? True


In [3]:
VENV_PY = PROJECT_ROOT / ".venv" / "Scripts" / "python.exe"
print("VENV python =", VENV_PY)
print("Existe ?", VENV_PY.exists())


VENV python = c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\.venv\Scripts\python.exe
Existe ? True


In [ ]:
import subprocess
from pathlib import Path

PROJECT_ROOT = Path(r"c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier")
VENV_PY      = PROJECT_ROOT / ".venv" / "Scripts" / "python.exe"

script_path = PROJECT_ROOT / "src" / "infer" / "superimages_infer.py"
splits_csv  = PROJECT_ROOT / "data" / "splits" / "super_images_3x3_folds.csv"
img_root    = PROJECT_ROOT / "data" / "super_images"
ckpt_root   = PROJECT_ROOT / "exp" / "results" / "super_images_convnext"

fold = 0
ckpt = ckpt_root / f"convnext_base_fold{fold}" / "best_model.pth"
out  = ckpt_root  / f"preds_convnext_fold{fold}.npz"

cmd = [
    str(VENV_PY), str(script_path),
    "--splits_csv", str(splits_csv),
    "--img_root",   str(img_root),
    "--ckpt",       str(ckpt),
    "--fold",       str(fold),
    "--backbone",   "convnext_base",
    "--out",        str(out),
    "--batch_size", "4",
    "--image_size", "1344",
]

print("Commande :", " ".join(cmd))

# ➜ On CAPTURE stdout/stderr pour voir l'erreur complète
result = subprocess.run(cmd, capture_output=True, text=True)

print("\n=== STDOUT ===")
print(result.stdout)

print("\n=== STDERR ===")
print(result.stderr)

print("\nReturn code:", result.returncode)


Commande : c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\.venv\Scripts\python.exe c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\src\infer\superimages_infer.py --splits_csv c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\data\splits\super_images_3x3_folds.csv --img_root c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\data\super_images --ckpt c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_convnext\convnext_base_fold0\best_model.pth --fold 0 --backbone convnext_base --out c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_convnext\preds_convnext_fold0.npz --batch_size 4 --image_size 1344


## Inference TResNet-XL (5 folds)

In [9]:
import timm
timm.list_models("tresnet*")


c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['tresnet_l', 'tresnet_m', 'tresnet_v2_l', 'tresnet_xl']

In [ ]:
from pathlib import Path
import os, sys, subprocess

# Racine du projet (celle où il y a src/, data/, exp/, etc.)
PROJECT_ROOT = Path(r"c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier")
os.chdir(PROJECT_ROOT)
print("CWD =", os.getcwd())

# Dossiers déjà définis avant
SUPERIMG_CSV = "data/splits/super_images_3x3_folds.csv"
SUPERIMG_ROOT = "data/super_images_3x3"

# Dossier des résultats TResNet
TRES_DIR = PROJECT_ROOT / "exp" / "results" / "super_images_tresnetxl"
print("TRES_DIR =", TRES_DIR)

def run_python(script, args):
    """
    Lance un script Python en sous-processus dans le même environnement.
    """
    cmd = [sys.executable, script] + args
    print("▶️ Commande :", " ".join(map(str, cmd)))
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("❌ ERREUR (code", result.returncode, ")")
        print(result.stderr)
    print("\n--- FIN DU SCRIPT ---\n")
    return result


In [15]:
PROJECT_ROOT = Path(r"c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier")
TRES_DIR = PROJECT_ROOT / "exp" / "results" / "super_images_tresnetxl"



for fold in range(5):
    ckpt = f"{TRES_DIR}/tresnet_xl.miil_in1k_fold{fold}/best_model.pth"
    out = f"{TRES_DIR}/preds_tresnetxl_fold{fold}.npz"

    run_python(
        "src/infer/superimages_infer.py",
        [
            "--splits_csv", SUPERIMG_CSV,
            "--img_root", SUPERIMG_ROOT,
            "--ckpt", ckpt,
            "--fold", str(fold),
            "--backbone", "tresnet_xl",
            "--out", out,
        ]
    )


▶️ Commande : c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\.venv\Scripts\python.exe src/infer/superimages_infer.py --splits_csv data/splits/super_images_3x3_folds.csv --img_root data/super_images_3x3 --ckpt c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_tresnetxl/tresnet_xl.miil_in1k_fold0/best_model.pth --fold 0 --backbone tresnet_xl --out c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_tresnetxl/preds_tresnetxl_fold0.npz


Exception in thread Thread-16 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 386: character maps to <undefined>


Device: cpu
Fold 0 â†’ 578 super-images
Loading checkpoint: c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_tresnetxl/tresnet_xl.miil_in1k_fold0/best_model.pth
Backbone utilisÃ© : tresnet_xl
[WARN] Super-image manquante pour '2019' â†’ data\super_images_3x3\data\super_images\2019_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '2036' â†’ data\super_images_3x3\data\super_images\2036_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1031' â†’ data\super_images_3x3\data\super_images\1031_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1041' â†’ data\super_images_3x3\data\super_images\1041_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1075' â†’ data\super_images_3x3\data\super_images\1075_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1790' â†’ data\super_images_3x3\data\s

Exception in thread Thread-18 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 386: character maps to <undefined>


Device: cpu
Fold 1 â†’ 607 super-images
Loading checkpoint: c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_tresnetxl/tresnet_xl.miil_in1k_fold1/best_model.pth
Backbone utilisÃ© : tresnet_xl
[WARN] Super-image manquante pour '209' â†’ data\super_images_3x3\data\super_images\209_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '2123' â†’ data\super_images_3x3\data\super_images\2123_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1008' â†’ data\super_images_3x3\data\super_images\1008_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1013' â†’ data\super_images_3x3\data\super_images\1013_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1964' â†’ data\super_images_3x3\data\super_images\1964_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1974' â†’ data\super_images_3x3\data\sup

...

Device: cpu
Fold 2 â†’ 557 super-images
Loading checkpoint: c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_tresnetxl/tresnet_xl.miil_in1k_fold2/best_model.pth
Backbone utilisÃ© : tresnet_xl
[WARN] Super-image manquante pour '2022' â†’ data\super_images_3x3\data\super_images\2022_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '2023' â†’ data\super_images_3x3\data\super_images\2023_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1011' â†’ data\super_images_3x3\data\super_images\1011_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1020' â†’ data\super_images_3x3\data\super_images\1020_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1916' â†’ data\super_images_3x3\data\super_images\1916_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '194' â†’ data\super_images_3x3\data\su

Exception in thread Thread-22 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 386: character maps to <undefined>


Device: cpu
Fold 3 â†’ 580 super-images
Loading checkpoint: c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_tresnetxl/tresnet_xl.miil_in1k_fold3/best_model.pth
Backbone utilisÃ© : tresnet_xl
[WARN] Super-image manquante pour '2092' â†’ data\super_images_3x3\data\super_images\2092_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '211' â†’ data\super_images_3x3\data\super_images\211_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '2112' â†’ data\super_images_3x3\data\super_images\2112_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '2129' â†’ data\super_images_3x3\data\super_images\2129_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1004' â†’ data\super_images_3x3\data\super_images\1004_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '1030' â†’ data\super_images_3x3\data\sup

Exception in thread Thread-24 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\Simon VANDERCOILDEN\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 257: character maps to <undefined>


Device: cpu
Fold 4 â†’ 559 super-images
Loading checkpoint: c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\exp\results\super_images_tresnetxl/tresnet_xl.miil_in1k_fold4/best_model.pth
Backbone utilisÃ© : tresnet_xl
[WARN] Super-image manquante pour '202' â†’ data\super_images_3x3\data\super_images\202_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '2049' â†’ data\super_images_3x3\data\super_images\2049_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '2088' â†’ data\super_images_3x3\data\super_images\2088_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '2096' â†’ data\super_images_3x3\data\super_images\2096_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '2106' â†’ data\super_images_3x3\data\super_images\2106_3x3.jpg, utilisation d'une image noire dummy.
[WARN] Super-image manquante pour '213' â†’ data\super_images_3x3\data\supe

## Ensemble final

In [7]:
run_python("src/infer/ensemble.py", [])

▶️ Commande : c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\.venv\Scripts\python.exe src/infer/ensemble.py

--- FIN DU SCRIPT ---



CompletedProcess(args=['c:\\Users\\Simon VANDERCOILDEN\\Desktop\\Scolaire\\UTC\\IM05\\TX01\\qv-pipe-classifier\\.venv\\Scripts\\python.exe', 'src/infer/ensemble.py'], returncode=2)

In [8]:
import os, glob

print("CWD =", os.getcwd())
print("Framewise npz :", glob.glob("exp/results/framewise_resnet18/*.npz"))

CWD = c:\Users\Simon VANDERCOILDEN\Desktop\Scolaire\UTC\IM05\TX01\qv-pipe-classifier\notebooks
Framewise npz : []
